# Preprocesamiento Datos ICTUS isquémico

## Librerías Necesarias

In [4]:
# Librerías Esenciales
# ==============================================================================
import pandas as pd
import numpy as np
import os
import UTILS_LT as LT
import pickle
import time

# Para usar los Núcleos de la CPU
# ==============================================================================
import multiprocessing
from sklearn import manifold
from sklearn.model_selection import train_test_split
# Machine Learning
# ==============================================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report 
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV,train_test_split, ParameterGrid,RepeatedKFold
from sklearn import metrics
from imblearn.under_sampling import NearMiss
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import umap

%matplotlib inline

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
import plotly.io as plt_io
import plotly.graph_objects as go

# Configuración Previa
pd.set_option('display.max_columns', None)

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

## Datos iniciales:

In [8]:
pd.read_spss(ruta).head()

,GDR_AP,Año,Edad,Edad_r,Sexo,Servicio,Estancia,Tipo_de_Ingreso,Tipalt_2,NumDiag,NumProc,NDA_r,NPA_r,Reingreso,Subcategoria_Diag_r,Diag1_r,Diag2_r,Diag3_r,Diag4_r,Diag5_r,Diag6_r,Diag7_r,Diag8_r,Diag9_r,Diag10_r,Diag11_r,Diag12_r,Diag13_r,Diag14_r,Proc1_r,Proc2_r,Proc3_r,Proc4_r,Proc5_r,Proc6_r,Proc7_r,Proc8_r,Proc9_r,Proc10_r,Proc11_r,Proc12_r,Proc13_r,Proc14_r,Proc15_r,Proc16_r,Proc17_r,Proc18_r,Proc19_r,Proc20_r,EPOC,CISQ,HTA,VALV,MCD,CONGENITAS,ARR,TNO_GI,TNO_HDROELEC,DM,IR,ANEMIA,TEP,FA,OBESIDAD,DISLIPEM,ICC,IRA,Trombosis_con_infarto_434.01,Oclusion_arteria_noespecif_con_infarto_434.91,Embolia_con_infarto_434.11,Estenosis_carotida_infarto_433.11,Estenosis_precerebral_multiple_infarto_433.31,Estenosis_arteria_precerebral_otra_infarto_433.81,Estenosis_arteria_vertebral_infarto_433.21,Estenosis_arteria_basilar_infarto_433.01,Estenosis_arteria_precerebral_neom_infarto_433.91,Epilepisa_todas,Fecha_de_nacimiento,Fecha_de_ingreso,Fecha_de_alta,Código_Postal,CCAA_Residencia,Servicio_2cat,Serv_2cat_rec,PRE_1
0,14.0,2010.0,82.0,75-84,Mujer,NRL,6.0,Urgente,Vivos,9.0,0.0,8-11 diag,0-3 proc,No,434.1,434.11,427.31,401.9,368.16,459.81,280.9,V10.02,909.3,351.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,Sí,No,No,No,Sí,No,No,No,No,Sí,No,Sí,No,No,No,No,No,No,Sí,No,No,No,No,No,No,No,1927-10-03,2010-03-24,2010-03-30,46000,Com Valenciana,NRL,Neurología,0.108230
1,14.0,2010.0,66.0,65-74,Hombre,NRL,4.0,Programado,Vivos,3.0,1.0,0-3 diag,0-3 proc,No,433.1,433.11,401.9,272.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,Sí,No,No,No,No,No,No,No,No,No,No,No,No,Sí,No,No,No,No,No,Sí,No,No,No,No,No,No,1943-06-06,2010-03-26,2010-03-30,11012,Andalucía,NRL,Neurología,0.014313
2,14.0,2010.0,53.0,45-54,Mujer,NRL,6.0,Urgente,Vivos,6.0,3.0,4-7 diag,0-3 proc,No,434.0,434.01,342.92,784.5,493.90,303.91,305.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.03,88.71,88.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Sí,Sí,No,No,No,No,No,No,No,No,No,1956-10-03,2010-03-24,2010-03-30,24004,Castilla-León,NRL,Neurología,0.015778
3,14.0,2010.0,82.0,75-84,Mujer,NRL,7.0,Urgente,Vivos,8.0,4.0,8-11 diag,4-7 proc,No,434.1,434.11,414.8,401.9,427.31,272.4,244.9,300.00,V58.69,NaN,NaN,NaN,NaN,NaN,NaN,87.03,88.91,88.71,88.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,Sí,No,No,No,Sí,No,No,No,No,No,No,Sí,No,Sí,No,No,No,No,Sí,No,No,No,No,No,No,No,1928-02-28,2010-03-23,2010-03-30,47005,Castilla-León,NRL,Neurología,0.072464
4,14.0,2010.0,72.0,65-74,Hombre,MIR,5.0,Urgente,Vivos,3.0,5.0,0-3 diag,4-7 proc,No,434.9,434.91,V15.29,433.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.03,87.44,88.91,88.41,89.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Sí,No,No,No,No,No,No,No,No,1937-06-01,2010-03-25,2010-03-30,10300,Extremadura,MIR,Med Interna,0.044938


## Importación y Preprocesado

In [9]:
variables_modelo_inicial = ['EDAD',
                            'SEXO',
                            'EXITUS',
                            'REINGRESO',
                            'HTA',
                            'DM',
                            'FA',
                            'DISLIPEM',
                            'ICC',
                            'ESTBASILAR',
                            'CISQUEM']

random_state = 44
seed = 44
ruta = r'C:/Users/LUIS/OneDrive - Universidad de Almeria/00 TORRECÁRDENAS/02 PROYECTOS/PROYECTO ICTUS/PROYECTO ICTUS (EN ACTIVO)/01_ ICTUS 2008-2012/01_GRD 14_ICTUS_v2_depurada.sav'
# lectura datos de la logística
data_logistica = pd.read_spss(ruta)

LT.normalize_columns(data_logistica, True)
data_logistica.rename(columns = {'TIPALT_2':'EXITUS'}, inplace = True)
variables_logistica = [x for x in data_logistica.columns]

dropear_columnas = [
    'FA',
    'GDR_AP',
    'ANYO',
    'SERVICIO',
    'NUMPROC',
    'SUBCATEGORIA_DIAG_R',
    'DIAG1_R',
    'DIAG2_R',
    'DIAG3_R',
    'DIAG4_R',
    'DIAG5_R',
    'DIAG6_R',
    'DIAG7_R',
    'DIAG8_R',
    'DIAG9_R',
    'DIAG10_R',
    'DIAG11_R',
    'DIAG12_R',
    'DIAG13_R',
    'DIAG14_R',
    'PROC1_R',
    'PROC2_R',
    'PROC3_R',
    'PROC4_R',
    'PROC5_R',
    'PROC6_R',
    'PROC7_R',
    'PROC8_R',
    'PROC9_R',
    'PROC10_R',
    'PROC11_R',
    'PROC12_R',
    'PROC13_R',
    'PROC14_R',
    'PROC15_R',
    'PROC16_R',
    'PROC17_R',
    'PROC18_R',
    'PROC19_R',
    'PROC20_R',
    'NUMDIAG',
    'FECHA_DE_NACIMIENTO',
    'FECHA_DE_INGRESO',
    'FECHA_DE_NACIMIENTO',
    'FECHA_DE_INGRESO',
    'FECHA_DE_ALTA',
    'CODIGO_POSTAL',
    'CCAA_RESIDENCIA',
    'SERVICIO_2CAT',
    'SERV_2CAT_REC',
    'PRE_1']


# Para SEXO --> Male=0, Female=1
# Pare el resto: --> No=0, Yes=1


data_logistica.drop(dropear_columnas, axis = 'columns', inplace = True)

variables_logistica = [x for x in data_logistica.columns]

copy = variables_logistica.copy()
copy.remove('ESTENOSIS_ARTERIA_BASILAR_INFARTO_433.01')
copy.remove('CISQ')
copy.append('ESTBASILAR')
copy.append('CISQUEM')

data_logistica.rename(columns = dict(zip(variables_logistica, copy)), inplace = True)
variables_logistica = [x for x in data_logistica.columns]

variables_cat_log = [x for x in data_logistica.select_dtypes(['category','object']).columns]
data_logistica[variables_cat_log] = data_logistica[variables_cat_log].astype('object')

data_logistica = data_logistica[~pd.isnull(data_logistica).any(axis=1)]

LT.binarizar_variables(data_logistica)

def auxiliar_binarizar_2(x: str) -> str:
    if x.startswith('V') or x.startswith('v'):
        return 0
    elif x.startswith('E') or x.startswith('e'):
        return 1  
    elif x.startswith('H') or x.startswith('h'):
        return 0
    elif x.startswith('M') or x.startswith('m'):
        return 1
    else:
        return x

data_logistica[['SEXO','EXITUS']] = data_logistica[['SEXO','EXITUS']].applymap(lambda x: auxiliar_binarizar_2(x))

# ========================================================
# ========================================================
EDAD_R_to_ordinal = {'24-34':1,
                     '35-44':2,
                     '45-54':3,
                     '55-64':4,
                     '65-74':5,
                     '75-84':6,
                     '85 o más':7
                     }

def data_transform_EDAD_R(df):
    df['EDAD_R'] = df['EDAD_R'].map(EDAD_R_to_ordinal)
    
    return df # No hace falta
# ========================================================

NDA_R_to_ordinal = {'0-3 diag':1,
                    '4-7 diag':2,
                    '8-11 diag':3,
                    '>=12 diag':4
                    }

def data_transform_NDA_R(df):
    df['NDA_R'] = df['NDA_R'].map(NDA_R_to_ordinal)
    
    return df # No hace falta

# ========================================================
NPA_R_to_ordinal = {'0-3 proc':1,
                    '4-7 proc':2,
                    '8-11 proc':3,
                    '12-15 proc':4,
                    '>= 16 proc':5
                    }

def data_transform_NPA_R(df):
    df['NPA_R'] = df['NPA_R'].map(NPA_R_to_ordinal)
    
    return df # No hace falta

# ========================================================

TIPO_INGRESO_to_ordinal = {'Urgente':1,
                           'Programado':2
                          }

def data_transform_TIPO_INGRESO(df):
    df['TIPO_DE_INGRESO'] = df['TIPO_DE_INGRESO'].map(TIPO_INGRESO_to_ordinal)
    
    return df # No hace falta

# ========================================================
data_transform_EDAD_R(data_logistica)
data_transform_NDA_R(data_logistica)
data_transform_NPA_R(data_logistica)
data_transform_TIPO_INGRESO(data_logistica)
# ========================================================

,EDAD,EDAD_R,SEXO,ESTANCIA,TIPO_DE_INGRESO,EXITUS,NDA_R,NPA_R,REINGRESO,EPOC,HTA,VALV,MCD,CONGENITAS,ARR,TNO_GI,TNO_HDROELEC,DM,IR,ANEMIA,TEP,OBESIDAD,DISLIPEM,ICC,IRA,TROMBOSIS_CON_INFARTO_434.01,OCLUSION_ARTERIA_NOESPECIF_CON_INFARTO_434.91,EMBOLIA_CON_INFARTO_434.11,ESTENOSIS_CAROTIDA_INFARTO_433.11,ESTENOSIS_PRECEREBRAL_MULTIPLE_INFARTO_433.31,ESTENOSIS_ARTERIA_PRECEREBRAL_OTRA_INFARTO_433.81,ESTENOSIS_ARTERIA_VERTEBRAL_INFARTO_433.21,ESTENOSIS_ARTERIA_PRECEREBRAL_NEOM_INFARTO_433.91,EPILEPISA_TODAS,ESTBASILAR,CISQUEM
0,82.0,6,1,6.0,1,0,3,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,66.0,5,0,4.0,2,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,53.0,3,1,6.0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
3,82.0,6,1,7.0,1,0,3,2,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
4,72.0,5,0,5.0,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186240,72.0,5,1,8.0,1,0,3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
186241,55.0,4,0,9.0,1,0,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
186242,88.0,7,0,10.0,1,0,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
186243,77.0,6,1,13.0,1,0,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [10]:
import pyreadstat

pyreadstat.write_sav(data_logistica, 'Data_preprocesada_ICTUS.sav')